# Spatial Synthesis using Insertion Loss Method

In [299]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy
from scipy import constants, integrate, signal
from scipy.signal import butter, freqz
import torch

%config InlineBackend.figure_format = 'svg'
plt.rcParams['lines.linewidth'] = 0.8

eps0 = constants.epsilon_0
mu0 = constants.mu_0

![Line](line.png)

In [300]:
L = 900e-6
W = 28e-6
G = 24e-6
H = 0.2e-6
S = 76e-6
D = 10e-6

In [301]:
L0 = 0.00508*(np.log(4*S/W) +  # inductance per unit length
              0.22* (W/S + S/W))

## Normalised Component Table

For 0.5[dB] ripple

In [302]:
lpf_05_prototype = pd.read_csv("Element_Values_for_Maximally_Flat_Low-Pass_Filter_Prototypes_05.csv")
lpf_05_prototype

,N,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11
0,1,2.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.4142,1.4142,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.0000,2.0000,1.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.7654,1.8478,1.8478,0.7654,1.0000,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.6180,1.6180,2.0000,1.6180,0.6180,1.0000,NaN,NaN,NaN,NaN,NaN
5,6,0.5176,1.4142,1.9318,1.9318,1.4142,0.5176,1.0000,NaN,NaN,NaN,NaN
6,7,0.4450,1.2470,1.8019,2.0000,1.8019,1.2470,0.4450,1.0000,NaN,NaN,NaN
7,8,0.3902,1.1111,1.6629,1.9615,1.9615,1.6629,1.1111,0.3902,1.0000,NaN,NaN
8,9,0.3473,1.0000,1.5321,1.8794,2.0000,1.8794,1.5321,1.0000,0.3473,1.0000,NaN
9,10,0.3129,0.9080,1.4142,1.7820,1.9754,1.9754,1.7820,1.4142,0.9080,0.3129,1.0


For 3[dB] ripple

In [303]:
lpf_3_prototype = pd.read_csv("Element_Values_for_Maximally_Flat_Low-Pass_Filter_Prototypes_05.csv")
lpf_3_prototype

,N,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11
0,1,2.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.4142,1.4142,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.0000,2.0000,1.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.7654,1.8478,1.8478,0.7654,1.0000,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.6180,1.6180,2.0000,1.6180,0.6180,1.0000,NaN,NaN,NaN,NaN,NaN
5,6,0.5176,1.4142,1.9318,1.9318,1.4142,0.5176,1.0000,NaN,NaN,NaN,NaN
6,7,0.4450,1.2470,1.8019,2.0000,1.8019,1.2470,0.4450,1.0000,NaN,NaN,NaN
7,8,0.3902,1.1111,1.6629,1.9615,1.9615,1.6629,1.1111,0.3902,1.0000,NaN,NaN
8,9,0.3473,1.0000,1.5321,1.8794,2.0000,1.8794,1.5321,1.0000,0.3473,1.0000,NaN
9,10,0.3129,0.9080,1.4142,1.7820,1.9754,1.9754,1.7820,1.4142,0.9080,0.3129,1.0


## Designing Low Pass Prototype Filter

Assume the DiCAD has 4 stages, then we can choose N = 9

In [304]:
stages = 4
N = 2*stages + 1

selected_element_values = lpf_05_prototype[lpf_05_prototype['N'] == 9]
selected_element_values = selected_element_values.values.tolist()[0]
selected_element_values.pop(0) # remove N

selected_element_values = [x for x in selected_element_values if not pd.isna(x)] # remove NaN

print(selected_element_values)

[0.3473, 1.0, 1.5321, 1.8794, 2.0, 1.8794, 1.5321, 1.0, 0.3473, 1.0]


1. Convert to stub problem with  unit elements at the ends of the filter
2. Applying the second Kuroda identity
3. Impedance and freq scaling

In [305]:
# converting to stub problem
def convert_to_stubs(selected_element_values):
    stub_impedances = []
    stub_impedances.append(1)
    for i, value in enumerate(selected_element_values):
        if (i % 2 == 0): # C
            stub_impedances.append(1/value)
        else: # L
            stub_impedances.append(value)
    stub_impedances.append(1)
    return stub_impedances

stub_impedances = convert_to_stubs(selected_element_values)
print(stub_impedances)

[1, 2.879355024474518, 1.0, 0.6526989099928203, 1.8794, 0.5, 1.8794, 0.6526989099928203, 1.0, 2.879355024474518, 1.0, 1]


In [ ]:
# use kuroda's identities on stub problem

def perform_kurodas(Z2, Z1):
    n_sq = 1 + Z2/Z1
    return Z2*n_sq, Z1*n_sq

def peform_kuroda_on_full_chain(selected_element_values):
    for i in range(len(selected_element_values)-1):
        selected_element_values[i], selected_element_values[i+1] = perform_kurodas(selected_element_values[i], selected_element_values[i+1])

$R_0 = 50[\Omega]$  
Scaling: $L\to R_0L$, $C\to\frac{C}{R_0}$, $R_s \to R_0$, $R_L\to R_0$

In [306]:
R0 = 50
Rs = R0

def scale_lpf(selected_element_values):
    scaled_lpf = []
    for i, value in enumerate(selected_element_values):
        if (i % 2 == 0): # C
            scaled_lpf.append(value/R0)
        else: # L
            scaled_lpf.append(value*R0)
    return scale_lpf